# Demo for paper "First Order Motion Model for Image Animation"

In [1]:
!cd C:\Users\Suyeon\Desktop\github\GIS_Hackathon\model

**Load driving video and source image**

In [2]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

source_image = imageio.imread('../model/data/images/02.png')
driving_video = imageio.mimread('../model/data/videos/4.mp4')


#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

**Create a model and load checkpoints**

config 폴더 없이 돌아감!

In [3]:
from demo_vox import load_checkpoints
generator, kp_detector = load_checkpoints(checkpoint_path='../model/checkpoint/vox-cpk.pth.tar',cpu=False)

**Perform image animation**

In [ ]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave('../model/output/generated01.mp4', [img_as_ubyte(frame) for frame in predictions])
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())

**In the cell above we use relative keypoint displacement to animate the objects. We can use absolute coordinates instead,  but in this way all the object proporions will be inherited from the driving video. For example Putin haircut will be extended to match Trump haircut.**

In [ ]:
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True)
HTML(display(source_image, driving_video, predictions).to_html5_video())

## Running on your data

**First we need to crop a face from both source image and video, while simple graphic editor like paint can be used for cropping from image. Cropping from video is more complicated. You can use ffpmeg for this.**

**Another posibility is to use some screen recording tool, or if you need to crop many images at ones use face detector(https://github.com/1adrianb/face-alignment) , see https://github.com/AliaksandrSiarohin/video-preprocessing for preprcessing of VoxCeleb.** 

In [ ]:
from demo_vox import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte

# load generator, kp_detector
generator, kp_detector = load_checkpoints(checkpoint_path='../model/checkpoint/vox-cpk.pth.tar',cpu=False)

# load image and video
source_image = imageio.imread('../model/demo/images/10.png')
driving_video = imageio.mimread('../model/demo/videos/1.mp4', memtest=False)


#Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

# make animation
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,
                             adapt_movement_scale=True)

#save resulting video
imageio.mimsave('../model/output/generated02.mp4', [img_as_ubyte(frame) for frame in predictions])

# display
HTML(display(source_image, driving_video, predictions).to_html5_video())